In [ ]:
# Please always run this cell at the beginning of your notebook!
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)  # please make sure you run this line so that your results are reproducible

WF_YEAR_FILE = 'modis_2018_United_Kingdom.csv'
WF_RESOLUTION_KM = 1.0  # set resolution to 4 km
WF_RESOLUTION_ARCSEC = 30  # 120 arcsec ~ 4 km

# geographic boundaries for UK. format: (lon_min, lat_min, lon_max, lat_max)
GEO_BOUND_UK = (-10, 49, 2, 61)

In [ ]:
from climada_petals.hazard import WildFire
from climada.hazard import Centroids
from climada.util.constants import ONE_LAT_KM
from climada.entity import ImpactFuncSet
from climada_petals.entity.impact_funcs.wildfire import ImpfWildfire
from climada.entity import LitPop
from climada.engine import Impact
from climada.engine import ImpactCalc
# You would get a FutureWarning here. You can ignore it.
# Please do not change the dependencies of CLIMADA. 
# They are very complex and may not work properly if you change them.

In [ ]:
# set the hazard resolution to 1 km. The provided data is at 1 km. Need a scaling factor of 1/1:
res_factor = 1./WF_RESOLUTION_KM

In [ ]:
# This function is provided for you.
def select_region(df, bound):
    """
    Select the region of interest from the dataframe.
    df: hazard dataframe
    bound: tuple in format (lon_min, lat_min, lon_max, lat_max)
    """
    return df[(df['longitude'] >= bound[0]) & 
              (df['latitude'] >= bound[1]) & 
              (df['longitude'] <= bound[2]) & 
              (df['latitude'] <= bound[3])]

In [ ]:
### load the year data
df_firms_year = pd.read_csv(os.path.join('../climada_petals/data', WF_YEAR_FILE))
# focus on California region
df_firms_year_uk = select_region(df_firms_year, GEO_BOUND_UK)
df_firms_year_uk

In [ ]:
wf_year_uk = WildFire()
wf_year_uk.set_hist_fire_seasons_FIRMS(df_firms_year_uk, centr_res_factor=res_factor)

### set_proba_fire_seasons default function calling

In [ ]:
### simulate.
num_fires_ca = int(wf_year_uk.n_fires.item())
ign_range = [num_fires_ca,num_fires_ca+1]

In [ ]:
wf_year_uk.ProbaParams.prop_proba = 0.21
# IMPORTANT: set keep_all_fires=True to keep all fires in the simulation.
# This will allow you to access the simulated event later.
wf_year_uk.set_proba_fire_seasons(n_fire_seasons=1, n_ignitions=ign_range, keep_all_fires=True)
print('The probabilistic season is appended to the historic season:', 
      "event_names", wf_year_uk.event_name,
      "event_ids", wf_year_uk.event_id)

In [ ]:
### plot the new simulated event. can be accessed through .event_id
wf_year_uk.plot_intensity(event=wf_year_uk.event_id[-1])

### Generate coordinates of fired points

In [ ]:
def simulate_fire_probability_array_from_ML_algorithms():
    # return an array of fire probabilities and corresponding coordinates
    res_deg = WF_RESOLUTION_ARCSEC / ONE_LAT_KM
    